# K-Nearest Neighbors Algorithm

- KNN is a lazy learner: it doesn't learn a discriminative function from the training data but memorizes the training data instead.
- Example of a nonparametric model subcategory known as instance-based learning
- KNN is susceptible to overfitting due to the curse of dimensionality
    - we can combat this by using feature selection and dimensionality reduction (but not regularization)
- The main steps of a KNN algorithm are:
    1. Choose the number k and the distance metric.
        - choosing the correct number k could mean the difference between overfitting and underfitting.
        - often, we choose a
    2. Find the k nearest neighbors of the sample that we want to classify.
    3. Assign the class label by majority vote.
        - in the case of a tie, we assign the class whose neighbors are the closest. If the different neighbors are equidistant from the sample, the algorithm will choose the class that comes first in the training dataset.